In [1]:
# Importing Libraries
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS 
import numpy as np
from matplotlib import pyplot as plt

In [2]:
#Loading the data
df = pd.read_csv("Data/Processed/citibike_2022_with_weather.csv")

/var/folders/wf/s0crnzgd66b7pdj9_7_b6mpw0000gn/T/ipykernel_19291/2851044232.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/Processed/citibike_2022_with_weather.csv")


# Aggregrating Data

In [3]:
df['value'] = 1

In [4]:
#Taking a 30% RANDOM sample
df_sample = df.sample(frac=0.3, random_state=42).copy()
df_sample.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge,value
15115901,F66F3FD176B86390,electric_bike,2022-08-03 09:32:03.967,2022-08-03 10:32:12.464,Delancey St & Eldridge St,5414.07,NaN,NaN,40.719440,-73.991804,40.670000,-73.990000,casual,2022-08-03,27.6,both,1
753105,9FA92A23DAA636C4,electric_bike,2022-09-16 08:02:48.369,2022-09-16 08:06:25.251,1 Ave & E 78 St,7020.09,E 88 St & 1 Ave,7235.13,40.771404,-73.953517,40.778301,-73.948813,member,2022-09-16,19.6,both,1
23821092,021C712DBEF40F84,classic_bike,2022-10-12 17:31:51.550,2022-10-12 17:46:15.185,St James Pl & Oliver St,5238.05,E 1 St & Bowery,5636.13,40.713079,-73.998512,40.724861,-73.992131,casual,2022-10-12,17.9,both,1
17525642,B23EC0300455C2AD,electric_bike,2022-08-14 15:53:17.790,2022-08-14 16:16:22.126,E 4 St & 2 Ave,5593.04,Columbia Heights & Cranberry St,4829.01,40.726239,-73.989637,40.700379,-73.995481,casual,2022-08-14,24.0,both,1
6397983,7E2CB79CE92BDB24,classic_bike,2022-05-10 21:09:56.750,2022-05-10 21:17:36.103,E 25 St & 2 Ave,6046.02,E 16 St & 5 Ave,6022.04,40.739126,-73.979738,40.737262,-73.992390,member,2022-05-10,15.4,both,1


In [17]:
#grouping only by station 
df_group = (
    df.groupby(['start_station_name', 'end_station_name'])['value']
    .count()
    .reset_index())
df_group.head()

,start_station_name,end_station_name,value
0,1 Ave & E 110 St,1 Ave & E 110 St,791
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,12


In [18]:
#Renaming columns
df_group.rename(columns={
    'start_station_name': 'start_station',
    'end_station_name': 'end_station',
    'value': 'trips'
}, inplace=True)

df_group.head()

,start_station,end_station,trips
0,1 Ave & E 110 St,1 Ave & E 110 St,791
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,12


In [19]:
#Check
print(df_group['trips'].sum())
print(df.shape)

29768714
(29838806, 17)


In [24]:
#Start Stations coordinates
start_lookup = (
    df[['start_station_name', 'start_lat', 'start_lng']]
    .dropna(subset=['start_station_name', 'start_lat', 'start_lng'])
    .drop_duplicates(subset=['start_station_name'])
    .rename(columns={
        'start_station_name': 'start_station'
    }))

In [25]:
#End Stations Coordinates
end_lookup = (
    df[['end_station_name', 'end_lat', 'end_lng']]
    .dropna(subset=['end_station_name', 'end_lat', 'end_lng'])
    .drop_duplicates(subset=['end_station_name'])
    .rename(columns={
        'end_station_name': 'end_station'
    }))

In [26]:
#Merging coordinates
df_map = df_group.merge(start_lookup, on='start_station', how='left')
df_map = df_map.merge(end_lookup, on='end_station', how='left')

In [27]:
#Dropping any rows missing coordinates
df_map = df_map.dropna(subset=['start_lat','start_lng','end_lat','end_lng'])
df_map.head()

,start_station,end_station,trips,start_lat,start_lng,end_lat,end_lng
0,1 Ave & E 110 St,1 Ave & E 110 St,791,40.792327,-73.9383,40.792327,-73.938300
1,1 Ave & E 110 St,1 Ave & E 18 St,2,40.792327,-73.9383,40.733812,-73.980544
2,1 Ave & E 110 St,1 Ave & E 30 St,4,40.792327,-73.9383,40.741444,-73.975361
3,1 Ave & E 110 St,1 Ave & E 39 St,1,40.792327,-73.9383,40.747140,-73.971130
4,1 Ave & E 110 St,1 Ave & E 44 St,12,40.792327,-73.9383,40.750020,-73.969053


In [37]:
#Limiting to top 300 routes
df_map = df_map.nlargest(300, 'trips').copy()
df_map.head()

,start_station,end_station,trips,start_lat,start_lng,end_lat,end_lng
294979,Central Park S & 6 Ave,Central Park S & 6 Ave,12041,40.765992,-73.976518,40.765909,-73.976342
147757,7 Ave & Central Park South,7 Ave & Central Park South,8541,40.766647,-73.979245,40.766741,-73.979069
782306,Roosevelt Island Tramway,Roosevelt Island Tramway,8213,40.757284,-73.953600,40.757284,-73.953600
548198,Grand Army Plaza & Central Park S,Grand Army Plaza & Central Park S,7287,40.764397,-73.973715,40.764397,-73.973715
800520,Soissons Landing,Soissons Landing,7275,40.692317,-74.014866,40.692317,-74.014866


# Initalizing Kepler.gl Map

In [39]:
m = KeplerGl(height=700, data={"NYC Top 300 Routes": df_map})
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'NYC Top 300 Routes':                             start_station                        end_stat…

In [40]:
#Saving Map
config = m.config

m.save_to_html(
    file_name="CitiBike_Top500_Kepler.html",
    read_only=False,
    config=config)

Map saved to CitiBike_Top500_Kepler.html!


In [13]:
import json
with open("kepler_config.json", "w") as f:
    json.dump(config, f)